# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
df = pd.read_csv ('/Users/edgarahmed/Desktop/ironhack/Semana_2/dia_3/labs/2.3-lab_df_calculation_and_transformation/your-code/orders.csv')
                      
df.head()

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75       17850  United Kingdom         22.00  
3       3.39       17850  United Kingdom         20.34  
4       3.39       17850  United Kingdom         20.34

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Agrupamos las dos columnas.

grouped = df.groupby('CustomerID')['amount_spent'].sum().reset_index()

# Display the grouped DataFrame
print(grouped)

      CustomerID  amount_spent
0          12346      77183.60
1          12347       4310.00
2          12348       1797.24
3          12349       1757.55
4          12350        334.40
...          ...           ...
4334       18280        180.60
4335       18281         80.82
4336       18282        178.05
4337       18283       2094.88
4338       18287       1837.28

[4339 rows x 2 columns]


In [4]:
#Ahora las ordenamos,

sorted_grouped = grouped.sort_values(by= 'amount_spent', ascending=False)

sorted_grouped

CustomerID  amount_spent
1690       14646     280206.02
4202       18102     259657.30
3729       17450     194550.79
3009       16446     168472.50
1880       14911     143825.06
...          ...           ...
4099       17956         12.75
3015       16454          6.90
1794       14792          6.20
3218       16738          3.75
685        13256          0.00

[4339 rows x 2 columns]

In [5]:
#Usamos el transcribe para ver los percentiles.


sorted_grouped.describe().T 

count          mean          std      min        25%       50%  \
CustomerID    4339.0  15299.936852  1721.889758  12346.0  13812.500  15299.00   
amount_spent  4339.0   2053.793018  8988.248381      0.0    307.245    674.45   

                   75%        max  
CustomerID    16778.50   18287.00  
amount_spent   1661.64  280206.02

In [6]:
#sacamos el 95% 

expenses_95th_percentile = sorted_grouped['amount_spent'].quantile(0.95)
vip_users = sorted_grouped[sorted_grouped['amount_spent'] >= expenses_95th_percentile]
vip_users = vip_users['CustomerID'].tolist()  # Get the 'Customer ID' values as a list


In [7]:
#Sacamos el 75%

expenses_75th_percentile = sorted_grouped['amount_spent'].quantile(0.75)
preferred_customers = sorted_grouped[
    (sorted_grouped['amount_spent'] >= expenses_75th_percentile) &
    (sorted_grouped['amount_spent'] < expenses_95th_percentile)
]
preferred_customers = preferred_customers['CustomerID'].tolist()  # Get the 'Customer ID' values as a list


In [8]:
print(preferred_customers[:10])

[13050, 12720, 15218, 17686, 13178, 16553, 13468, 14110, 14049, 17049]


In [9]:
print (vip_users[:10])

[14646, 18102, 17450, 16446, 14911, 12415, 14156, 17511, 16029, 12346]


In [10]:
def custumer_status(x):
    if x in vip_users:
        return 'VIP'
    elif x in preferred_customers:
        return 'Preferred'
    else:
        return 'Regular'

In [11]:
df['Client Status'] = df['CustomerID'].apply(custumer_status)

In [12]:
print (df['Client Status'])  #Podemos ver que los ha organizado

0         Preferred
1         Preferred
2         Preferred
3         Preferred
4         Preferred
            ...    
397919      Regular
397920      Regular
397921      Regular
397922      Regular
397923      Regular
Name: Client Status, Length: 397924, dtype: object


In [13]:
df.shape #la longitud del df es igual, con una columna nueva.

(397924, 15)

In [14]:
df.head()

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent Client Status  
0       2.55       17850  United Kingdom         15.30     Preferred  
1       3.39       17850  United Kingdom         20.34     Preferred  
2       2.75       17850  United Kingdom         22.00     Preferred  
3       3.39       17850  United Kingdom         20.34     Preferred  
4       3.39       17850  United Kingdom         20.34     Preferred

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [15]:
#Definimos
data = ('Client Status','Country')
customer_data = df  

In [16]:
#nuevas variables para sacar la info.
unique_client_status = customer_data['Client Status'].unique()
unique_countries = customer_data['Country'].unique()


In [17]:
# Identificar los VIP clients
vip_clients = customer_data[customer_data['Client Status'] == 'VIP']

In [18]:
# Count VIP clients by country
vip_clients_by_country = vip_clients['Country'].value_counts()

In [19]:
# Find the country with the most VIP clients
most_vip_country = vip_clients_by_country.idxmax()
most_vip_count = vip_clients_by_country.max()


In [20]:
print(f"The country with the most VIP clients is {most_vip_country} with {most_vip_count} VIP clients.")

The country with the most VIP clients is United Kingdom with 84185 VIP clients.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
# your code here

In [22]:
preferred_clients = customer_data[customer_data['Client Status'] == 'Preferred']

preferred_clients[:10]

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   
5           5     536365     22752  2010     12    3     8   
6           6     536365     21730  2010     12    3     8   
7           7     536366     22633  2010     12    3     8   
8           8     536366     22632  2010     12    3     8   
9           9     536367     84879  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   
5         set 7 babushka nesting boxes         2  2010-12-01 08:26:00   
6    glass star frosted t-light holder         6  2010-12-01 08:26:00   
7               hand warmer union jack         6  2010-12-01 08:28:00   
8            hand warmer red polka dot         6  2010-12-01 08:28:00   
9        assorted colour bird ornament        32  2010-12-01 08:34:00   

   UnitPrice  CustomerID         Country  amount_spent Client Status  
0       2.55       17850  United Kingdom         15.30     Preferred  
1       3.39       17850  United Kingdom         20.34     Preferred  
2       2.75       17850  United Kingdom         22.00     Preferred  
3       3.39       17850  United Kingdom         20.34     Preferred  
4       3.39       17850  United Kingdom         20.34     Preferred  
5       7.65       17850  United Kingdom         15.30     Preferred  
6       4.25       17850  United Kingdom         25.50     Preferred  
7       1.85       17850  United Kingdom         11.10     Preferred  
8       1.85       17850  United Kingdom         11.10     Preferred  
9       1.69       13047  United Kingdom         54.08     Preferred

In [23]:
combined_clients = pd.concat([vip_clients, preferred_clients])


In [24]:
combined_clients_by_country = combined_clients['Country'].value_counts()


In [25]:
most_combined_country = combined_clients_by_country.idxmax()
most_combined_count = combined_clients_by_country.max()

In [26]:
print(f"The country with the most VIP and Preferred clients combined is {most_combined_country} with {most_combined_count} clients.")


The country with the most VIP and Preferred clients combined is United Kingdom with 221635 clients.
